In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_dir = './train/images'
label_dir = './train/labels'

labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])

In [ ]:
for img_file in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_file)
    label_path = os.path.join(label_dir, img_file.replace(".jpg", ".txt"))
    
    image = cv2.imread(img_path)
    if image is None:
        continue
    
    with open(label_path, "r") as f:
        label = f.readline().strip().split()[1:]
        label = [float(x) for x in label]
        
        x_points = label[::2]
        y_points = label[1::2]
        
        img_h, img_w, _ = image.shape
        x_points = [int(x * img_w) for x in x_points]
        y_points = [int(y * img_h) for y in y_points]
        
        x_min = min(x_points)
        x_max = max(x_points)
        y_min = min(y_points)
        y_max = max(y_points)
        
        labels_dict['filepath'].append(img_path)
        labels_dict['xmin'].append(x_min)
        labels_dict['xmax'].append(x_max)
        labels_dict['ymin'].append(y_min)
        labels_dict['ymax'].append(y_max)

In [ ]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

In [ ]:
image_path = list(df['filepath'])

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    load_image = tf.keras.utils.load_img(image,target_size=(224,224))
    load_image_arr = tf.keras.utils.img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [ ]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
inception_resnet = tf.keras.applications.InceptionResNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
inception_resnet.trainable = False

In [ ]:
headmodel = inception_resnet.output
headmodel = tf.keras.layers.Flatten()(headmodel)
headmodel = tf.keras.layers.Dense(500,activation="relu")(headmodel)
headmodel = tf.keras.layers.Dense(250,activation="relu")(headmodel)
headmodel = tf.keras.layers.Dense(4,activation='sigmoid')(headmodel)

In [ ]:
model = tf.keras.Model(inputs=inception_resnet.input,outputs=headmodel)

In [ ]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

In [ ]:
import tensorflow as tf

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_freq_epochs=100, filepath='model_{epoch:03d}.keras'):
        super(CustomModelCheckpoint, self).__init__()
        self.save_freq_epochs = save_freq_epochs
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq_epochs == 0:
            save_path = self.filepath.format(epoch=epoch + 1)
            self.model.save(save_path)
            print(f'\nModel saved to {save_path}\n')

checkpoint_callback = CustomModelCheckpoint(save_freq_epochs=100, filepath='model_{epoch:03d}.keras')

tfb = tf.keras.callbacks.TensorBoard('object_detection')

history = model.fit(
    x=x_train, y=y_train, 
    batch_size=10, epochs=300,
    validation_data=(x_test, y_test), 
    callbacks=[tfb, checkpoint_callback]
)

In [ ]:
model = tf.keras.models.load_model('./model_300.keras')

In [ ]:
test_loss = model.evaluate(x_test, y_test)
print(f"Test MSE: {test_loss}")

In [ ]:
def calculate_iou(box1, box2):
    x1_max = min(box1[1], box2[1])
    x1_min = max(box1[0], box2[0])
    y1_max = min(box1[3], box2[3])
    y1_min = max(box1[2], box2[2])
    
    inter_area = max(0, x1_max - x1_min) * max(0, y1_max - y1_min)
    
    box1_area = (box1[1] - box1[0]) * (box1[3] - box1[2])
    box2_area = (box2[1] - box2[0]) * (box2[3] - box2[2])
    
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

In [ ]:
def calculate_map(ground_truths, predictions, iou_threshold=0.5):
    """Calculate mean Average Precision (mAP) at a specific IoU threshold."""
    aps = []
    for pred_box, gt_box in zip(predictions, ground_truths):
        iou = calculate_iou(gt_box, pred_box)
        if iou >= iou_threshold:
            aps.append(1.0)
        else:
            aps.append(0.0)

    return np.mean(aps)

In [ ]:
def calculate_ar(predictions, ground_truths, iou_thresholds=[0.5, 0.75]):
    recalls = []
    for iou_threshold in iou_thresholds:
        all_recalls = []
        for pred_box, gt_box in zip(predictions, ground_truths):
            iou = calculate_iou(pred_box, gt_box)
            recall = iou >= iou_threshold
            all_recalls.append(recall)

        ar = np.mean(all_recalls)
        recalls.append(ar)
    
    AR = np.mean(recalls)
    return AR

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import PrecisionRecallDisplay

def plot_precision_recall_curve(y_true, y_pred, label='PR Curve'):
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
    avg_precision = average_precision_score(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    plt.plot(recalls, precisions, marker='.', label=f'{label} (AP={avg_precision:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.show()

In [ ]:
y_pred = model.predict(x_test)

denorm_factors = np.array([x_test.shape[2], x_test.shape[2], x_test.shape[1], x_test.shape[1]])
y_pred_denorm = y_pred * denorm_factors

In [ ]:
y_test_denorm = y_test * denorm_factors

predictions = [pred_box for pred_box in y_pred_denorm]
ground_truths = [gt_box for gt_box in y_test_denorm]

In [ ]:
predictions = np.array(predictions)
ground_truths = np.array(ground_truths)

In [ ]:
mAP = calculate_map(predictions, ground_truths)
AR = calculate_ar(predictions, ground_truths)
ious = [calculate_iou(pred, true) for pred, true in zip(y_pred, y_test)]
mean_iou = np.mean(ious)

In [ ]:
print(f"Mean Average Precision (mAP): {mAP:.2f}")
print(f"Average Recall (AR): {AR:.2f}")
print(f"Mean IoU: {mean_iou}")

In [ ]:
def plot_iou_histogram(ious):
    plt.figure(figsize=(8, 6))
    plt.hist(ious, bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('IoU Score')
    plt.ylabel('Frequency')
    plt.title('IoU Score Distribution')
    plt.show()

plot_iou_histogram(ious)

In [ ]:
def plot_map_vs_iou(iou_thresholds, map_values):
    plt.figure(figsize=(8, 6))
    plt.plot(iou_thresholds, map_values, marker='o', color='blue')
    plt.xlabel('IoU Threshold')
    plt.ylabel('mAP')
    plt.title('mAP vs. IoU Threshold')
    plt.show()

iou_thresholds = np.linspace(0.5, 0.95, 10)
map_values = [calculate_map(ground_truths, predictions, threshold) for threshold in iou_thresholds]
plot_map_vs_iou(iou_thresholds, map_values)

In [ ]:
def plot_cumulative_iou_curve(ious):
    sorted_ious = sorted(ious)
    cumulative = np.cumsum(sorted_ious) / len(ious)
    plt.figure(figsize=(8, 6))
    plt.plot(sorted_ious, cumulative, color='purple')
    plt.xlabel('IoU Threshold')
    plt.ylabel('Cumulative Distribution')
    plt.title('Cumulative IoU Curve')
    plt.show()

plot_cumulative_iou_curve(ious)

In [ ]:
def precision_recall_f1(ious, threshold=0.5):
    true_positives = sum(iou > threshold for iou in ious)
    false_positives = len(ious) - true_positives
    false_negatives = len(y_test) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1_score

In [ ]:
precision, recall, f1_score = precision_recall_f1(ious)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1_score:.2f}")

In [ ]:
def object_detection(path):
    image = tf.keras.utils.load_img(path)
    image = np.array(image,dtype=np.uint8)
    image1 = tf.keras.utils.load_img(path,target_size=(224,224))
    image_arr_224 = tf.keras.utils.img_to_array(image1)/255.0
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)
    coords = model.predict(test_arr)
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

In [ ]:
import random

def get_random_image_path(directory):
    files = os.listdir(directory)
    
    files = [f for f in files if os.path.isfile(os.path.join(directory, f))]
    
    return os.path.join(directory, random.choice(files))

In [ ]:
path = get_random_image_path(img_dir)
print(path)
image, cods = object_detection(path)

plt.figure(figsize=(10,8))
plt.imshow(image)
plt.show()

In [ ]:
clean_path = path.strip("'")
matching_row = df[df['filepath'] == clean_path]
if not matching_row.empty:
    extracted_values = matching_row.iloc[0, 1:].tolist()
    result = ', '.join(map(str, extracted_values))
    print (result)
    values = result.split(', ')
else:
    print("No matching row found.")
xmin,xmax,ymin,ymax = map(int, values)
img = cv2.imread(path)
cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 3)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()